In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import scipy.io as sp
import logging

#traitement mat
cancercell=sp.loadmat('breastw.mat')
X = cancercell['X']
y = cancercell['y']
#print(cancercell)
y=np.reshape(y,(683,))
#Séparation des données d'apprentissage et des données tests
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, stratify=y)
#fin traitement mat

plt.figure(figsize=(18,5))

plt.subplot(1, 3, 3)
plt.title('Receiver Operating Characteristic')
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.title('ROC Curve of kNN')

logging.basicConfig(level=logging.DEBUG)

i=0
MC=np.zeros((48,2))
for k in [3, 4, 5, 6, 7, 8]:
    for w in ['uniform','distance']:
        for a in ['auto','ball_tree','kd_tree','brute']:

            MC[i,0]=[k,w,a]
            #algo knn
            model = KNeighborsClassifier(n_neighbors=k, weights=w, algorithm=a)
            model.fit(X_train,y_train)
            prediction= model.predict(X_test)
            model_PCA = PCA(n_components=2)
            data_reduced = model_PCA.fit_transform(X_test)
            #fin algo knn

            logging.debug('i=%d : k=%d, w=%s, a=%s' % (i, k, w, a))

            #print(model.predict_proba(X_test))
            y_scores = model.predict_proba(X_test)

            #graphique
            #  ROC curve
            fpr, tpr, threshold = roc_curve(y_test, y_scores[:, 1])
            roc_auc = auc(fpr, tpr)
            MC[i,1] = roc_auc

            logging.debug('i=%d : AUC=%f' % (i, roc_auc))
            #plt.plot(fpr, tpr, 'b', label = 'AUC = %0.4f' % roc_auc)
            #plt.plot(fpr, tpr, 'b', label = 'AUC {}'.format(k) & '%0.4f' % roc_auc)
            #fin graphique
            i += 1 
    
print(MC)    

"""""
plt.subplot(1, 3, 1)
plt.scatter(data_reduced[:,0],data_reduced[:,1],c=prediction)
plt.title("Résultat pred")
maline = mpatches.Patch(color='yellow', label='maline')
benine = mpatches.Patch(color='purple', label='benine')
plt.legend(handles=[maline,benine])

plt.subplot(1, 3, 2)
plt.scatter(data_reduced[:,0],data_reduced[:,1],c=y_test)
plt.title("Résultat vrai")
maline = mpatches.Patch(color='yellow', label='maline')
benine = mpatches.Patch(color='purple', label='benine')
plt.legend(handles=[maline,benine])

plt.show()
"""

No handles with labels found to put in legend.


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
result = confusion_matrix(y_test, prediction)
print("Matrice de confusion :")
print(result)
result1 = classification_report(y_test, prediction)
print("Classification Report:",)
print (result1)
result2 = accuracy_score(y_test,prediction)
print("Accuracy :",result2)

Matrice de confusion :
[[216   6]
 [  5 115]]
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.97      0.98       222
           1       0.95      0.96      0.95       120

    accuracy                           0.97       342
   macro avg       0.96      0.97      0.96       342
weighted avg       0.97      0.97      0.97       342

Accuracy : 0.9678362573099415
